In [1]:
import git
g = git.cmd.Git('/root/Projects/Assignment-Projects')
g.pull()

'Already up-to-date.'

In [1]:
from Sparker.SparkLogProcesser.SparkLogFileHandler import setSparkContext
from pyspark import SparkContext
sc = SparkContext() 
setSparkContext(sc)

In [2]:
from Sparker.Logic.LogicTests import *
day2Logs = readParsedLogsFromHDFS(entireDayParsedLogsFolder2)

NoExistsOnUCSC/Parsed/2016-09-28_parsed has been read by 2017-05-19 04:46:25.370486


In [3]:
from MainSrc.PythonVersionHandler import *
from paths import *

In [4]:
from Sparker.Logic.LogicTests import *
from Sparker.Logic.Trainer import *
#trainPairWiseData(data) 

In [5]:
def printSeparater(c = 3):
    for n in range(c):
        print_('#' * 88)

In [6]:
from sklearn import svm

def evaluateModelOnData(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict(p.features)))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    return labelsAndPreds

def trainPairWiseData(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    print_('Training...')
    model = SVMWithSGD.train(data, iterations=100)
    print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())##
    print_('The learned weights:\n' + str(model.weights).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnData(model, data, dataName, modelName)
    return model

def runTrainingExperiment(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    model = trainPairWiseData(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnData(model, testData, 'testData', modelName)

In [7]:
def evaluateModelOnDataSK(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict(p.features)))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    return labelsAndPreds

def trainPairWiseDataSK(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    model = svm.SVC()#kernel =  'poly', degree = 3) #[:size][:size]
    size = 5000
    X =  data.map(lambda p: p.features).collect()
    Y =  data.map(lambda p: p.label).collect()
    print_('Training...')
    model.fit(X, Y)
    print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())##
   # print_('The learned weights:\n' + str(model.coef_).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnDataSK(model, data, dataName, modelName)
    return model

def runTrainingExperimentSK(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    model = trainPairWiseDataSK(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnDataSK(model, testData, 'testData', modelName)

In [23]:
from DeepLearningToRank.DeepDataHandler import readTrainDataFromPickle
#from DeepLearningToRank.DeepTrainer import *

def featureFilter(trainData):
    return trainData.map(lambda x: LabeledPoint(x.label, list(x.features[1:4]) + list(x.features[5:6]) + list(x.features[7:-2])))

def runExperiment(trainData, modelName = 'Model'):
    printSeparater(1)
    testData, trainData = trainData.randomSplit(weights=[0.3, 0.7])
    #testData, trainData = trainData[:int(30*len(trainData)/100)], trainData[int(30*len(trainData)/100):]SK
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
    print_('The experiment for', modelName, 'has finished by', nowStr())
    
def trainPairWiseDataTestKeyword2(keyword, location = 'HDF.S'):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    if location == 'HDFS':
        outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        trainDataFile = joinPath(outputFolder, inputName + '_' + keyword + '_TrainData')
        trainData = readTrainDataFromHDFS(trainDataFile)
    else:
        trainDataFile = joinPath(textTrainDataFolder, inputName + '_' + keyword + '_TrainData.txt')
        trainData = readTrainDataFromPickle(trainDataFile)
        trainData = sc_().parallelize(trainData)
        trainData = trainData.map(lambda x: LabeledPoint(x[0], x[1]))
    print_(trainData.first())
    preprocessing = scaleTrainData# normalizeTrainData#
    trainData = preprocessing(trainData)
    runExperiment(trainData, modelName = 'ExtendedModel')
    trainData = featureFilter(trainData)
    runExperiment(trainData, modelName = 'FilteredModel')
    printSeparater(2)

In [24]:
trainPairWiseDataTestKeyword2('iphone 6')

########################################################################################
########################################################################################
/soe/cicekm/Projects/offlineData/all_day_iphone_6_TrainData.txt has been read successfully by 2017-05-19 05:25:41.983985
(1.0,[0.0,15.0,100.0,-4001.0,1.0,1.0,0.0,1.0,0.0,1.0,75.3,-780.0])
835692 instances have been scaled by 2017-05-19 05:25:48.258672
########################################################################################
Training...

 ExtendedModel has been trained on trainData by 2017-05-19 05:26:41.854098
The learned weights:
[-0.0135669704447, 0.693874282493, 0.0365743330955, 0.0167163902482, 0.0352732652536, 0.180662009167, -0.656961146254, 0.165403902962, 0.293123576531, 0.511020110259, 1.1648170067, -0.212257484789]


ExtendedModel has been evaluated on trainData by 2017-05-19 05:26:50.599279
The result accuracy is %89.993


ExtendedModel has been evaluated on testData by 2017-05-19 05:2

In [13]:
import pickle
def savePairWisetrainData(keyword):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    outputFolder = joinPath(HDFSDataFolder, 'Day1_' + keyword + '_Data')
    dataTypes = ['_TrainData', '_labeledPairs', '_journey_products']
    for typ in dataTypes:
        fileName = inputName + '_' + keyword + typ
        trainDataFile = joinPath(outputFolder, fileName)
        trainData = sc_().textFile(trainDataFile)
        f = joinPath('/root/Projects/TrainData6', fileName)
        trainData = trainData.collect()
        fp = open(f, "wb")   #Pickling
        pickle.dump(trainData, fp)
        print_(f, 'has been saved successfully by', nowStr())

In [10]:
def trainPairWiseDataTest():
    keywords = ['iphone 6', 'jant', 'nike air max', 'tv unitesi', 'kot ceket', 'camasir makinesi', 'bosch']
    for keyword in keywords:
        #trainPairWiseDataTestKeyword2(keyword)readTrainDataFromHDFS
        savePairWisetrainData(keyword)

In [11]:
def main():
    printSeparater()
    print_('%s:' % nowStr(), 'Running on osldevptst02:Jupyter...')

    trainPairWiseDataTest()

    print_('%s:' % nowStr(), 'DONE')
    printSeparater()


In [15]:
main()

########################################################################################
########################################################################################
########################################################################################
2017-05-09 15:09:16.404133: Running on osldevptst02:Jupyter...
########################################################################################
########################################################################################
/root/Projects/TrainData6/all_day_tv_unitesi_TrainData has been saved successfully by 2017-05-09 15:09:35.835378
/root/Projects/TrainData6/all_day_tv_unitesi_labeledPairs has been saved successfully by 2017-05-09 15:09:56.224687
/root/Projects/TrainData6/all_day_tv_unitesi_journey_products has been saved successfully by 2017-05-09 15:09:58.585784
2017-05-09 15:09:58.589724: DONE
########################################################################################
#####################